In [38]:
from functools import partial
from cjm_psl_utils.core import download_file, file_extract
from cjm_torchvision_tfms.core import ResizeMax, PadSquare, CustomRandomIoUCrop


import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms
import os
import cv2
from torch.utils.data import DataLoader
from torch.utils.data import Dataset
import random
import torch.optim as optim
import matplotlib.pyplot as plt
torch.manual_seed(42)
import numpy as np
from PIL import Image
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchvision.datasets import MNIST
import cv2 as cv
# from google.colab.patches import cv2_imshow
import pytorch_lightning as pl
from pytorch_lightning import LightningDataModule, LightningModule, Trainer
from torch.autograd import Variable
from pytorch_lightning import loggers as pl_loggers
import torch.nn.functional as F


# Import torchvision dependencies
import torchvision
torchvision.disable_beta_transforms_warning()
from torchvision.tv_tensors import BoundingBoxes
from torchvision.utils import draw_bounding_boxes
import torchvision.transforms.v2  as transforms

import sys
import torch
import torchvision
import numpy as np

from torch.utils.data import DataLoader

from dataset import CardDlayingDataset, final_transforms, custom_collate
from model import CenterNet
from loss import focal_loss, get_predictions_from_head
from inferance import gather_feature, topKscoresPerBatch, pool_nms

In [39]:
def gather_feature(fmap, index, mask=None, use_transform=False):
    if use_transform:
        # change a (N, C, H, W) tenor to (N, HxW, C) shape
        batch, channel = fmap.shape[:2]
        fmap = fmap.view(batch, channel, -1).permute((0, 2, 1)).contiguous()
    print('----------> ', fmap.shape)
    dim = fmap.size(-1)
    index = index.unsqueeze(len(index.shape)).expand(*index.shape, dim)
    print('----------> ', index)
    fmap = fmap.gather(dim=1, index=index)


    if mask is not None:
        # this part is not called in Res18 dcn COCO
        mask = mask.unsqueeze(2).expand_as(fmap)
        fmap = fmap[mask]
        fmap = fmap.reshape(-1, dim)
    return fmap



In [40]:
index = np.random.randint(0, 64, (2, 5))
index = torch.tensor(index)

size = torch.rand(2, 2, 64, 64)

print(index.shape, size.shape)

fmap = gather_feature(size, index, use_transform=True)

print(fmap.shape)



torch.Size([2, 5]) torch.Size([2, 2, 64, 64])
---------->  torch.Size([2, 4096, 2])
---------->  tensor([[[27, 27],
         [ 3,  3],
         [ 6,  6],
         [51, 51],
         [63, 63]],

        [[14, 14],
         [15, 15],
         [28, 28],
         [30, 30],
         [ 6,  6]]])
torch.Size([2, 5, 2])


In [41]:
import torch
from torchvision.ops import box_iou

# Bounding boxes in format [x1, y1, x2, y2]
boxes1 = torch.tensor([[0, 0, 1, 1], [0, 0, 2, 1]])
boxes2 = torch.tensor([[0, 0, 1, 1]])

iou = box_iou(boxes1, boxes2)
print(iou)

tensor([[1.0000],
        [0.5000]])


In [42]:
test = torch.tensor([1, 7, 5, 4])

bbox = torch.tensor([[1, 1, 1, 1],
                     [2, 2, 2, 2],
                     [3, 3, 3, 3],
                     [4, 4, 4, 4]])

print('initial tensor:', test)

test, indices = test.sort()

print('sorted tensor:', test)
print('indices:', indices)

bbox = bbox[indices, ::] 

print('reorganised bbox:')
for zz in bbox:
    print(zz)

initial tensor: tensor([1, 7, 5, 4])
sorted tensor: tensor([1, 4, 5, 7])
indices: tensor([0, 3, 2, 1])
reorganised bbox:
tensor([1, 1, 1, 1])
tensor([4, 4, 4, 4])
tensor([3, 3, 3, 3])
tensor([2, 2, 2, 2])


In [51]:
from torchvision.ops import box_iou

boxes1 = torch.tensor([[152.1973, 122.4850, 165.6881, 164.4115]])
boxes2 = torch.tensor([[151.6000, 123.6000, 167.6000, 149.6000]])

iou = box_iou(boxes1, boxes2)
print(iou)

tensor([[0.5560]])
